In [15]:
import pandas as pd
from IPython.display import Markdown

from tb_incubator.constants import set_project_base_path
from tb_incubator.constants import compartments, infectious_compartments, model_times, age_strata, indicator_names
from tb_incubator.model import build_model
from tb_incubator.plotting import set_plot_label, display_plot, plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info, get_param_table

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")


## Model construction

In [16]:
param_info = load_param_info()
params = param_info["value"]

In [ ]:
model, desc = build_model(
    compartments,
    infectious_compartments,
    age_strata,
    params,
    model_times
)
Markdown(desc)

## Parameters

In [18]:
calib_params = {'contact_rate': 2.5005249495731374,
               'screening_inflection_time': 2015.4973243240213,
               'screening_scaleup_shape': 0.06997698059772811,
               'self_recovery_rate': 0.12085224978486633,
               'time_to_screening_end_asymp': 3.2205030858785073}

In [19]:
model.run(params | calib_params)

In [ ]:
param_text = "The parameters used in this model are listed below. Please note that most of the values are currently arbitrary, as we have not yet performed calibration at this stage."
Markdown(param_text)

In [ ]:
fixed_param_table = get_param_table(param_info)
Markdown(fixed_param_table.to_markdown())

## Results

In [22]:
targets = load_targets()
outs = model.get_derived_outputs_df()

### Size of each compartment

In [ ]:
fig = outs[[f"comp_size_{c}" for c in compartments]].plot.area()
set_plot_label(fig, indicator_names, "Compartment size")
display_plot(fig, "comp_size", "svg")
#fig

### TB Notifications

In [ ]:
fig = plot_model_vs_actual(outs, 
                     targets["notif"],
                     "notification",
                     "Notification",
                     "Modelled vs Data")
fig.add_scatter(x=targets["new_cases"].index, y=targets["new_cases"], mode="markers", name="Actual Data (NTCP)")
display_plot(fig, "notification", "svg")

### TB incidence, prevalence, and deaths

In [ ]:
fig = outs[["incidence", "prevalence", "mortality"]].plot()
set_plot_label(fig, indicator_names, "value")
display_plot(fig, "inc_prev_mort", "svg")

In [ ]:
fig = plot_model_vs_actual(outs, 
                     targets["prevalence"],
                     "prevalence",
                     "Prevalence",
                     "Modelled vs Data (WHO)")

display_plot(fig, "prevalence", "svg")

### Notification per incidence

In [ ]:
actual_data = targets["notif"] / targets["incidence_num"] * 100
fig = plot_model_vs_actual(outs, 
                     actual_data,
                     "notification_per_incidence",
                     "Notification per incidence",
                     "Modelled vs Data (WHO)")

display_plot(fig, "notification_per_incidence", "svg")

### Percentage of latent TB infection

In [ ]:
fig = outs["percentage_latent"].plot()
set_plot_label(fig, indicator_names, "value")
display_plot(fig, "percentage_latent", "svg")

## References